# CSIRO Image2Biomass Prediction with Lightning ⚡

Competition: https://www.kaggle.com/competitions/csiro-biomass

Author: Based on https://github.com/Borda/kaggle_image-classify

In [ ]:
# !pip install -q pytorch-lightning torchmetrics timm

In [ ]:
import os
import glob
import random
from typing import Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

import pytorch_lightning as pl
import torchmetrics
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
print(f"PyTorch: {torch.__version__}")
print(f"Lightning: {pl.__version__}")
print(f"TIMM: {timm.__version__}")
print(f"Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

pl.seed_everything(42)

## Explore Data

In [ ]:
PATH_DATA = '/kaggle/input/csiro-biomass'
PATH_TRAIN_CSV = os.path.join(PATH_DATA, 'train.csv')
PATH_TRAIN_IMG = os.path.join(PATH_DATA, 'train')
PATH_TEST_IMG = os.path.join(PATH_DATA, 'test')

df = pd.read_csv(PATH_TRAIN_CSV)
print(f"Dataset size: {df.shape}")
df.head()

In [ ]:
TARGET_COLS = [c for c in df.columns if c not in ['image_id', 'Image']]
print(f"Target columns: {TARGET_COLS}")
print(f"Number of targets: {len(TARGET_COLS)}")

## Plot target distribution

In [ ]:
# Exclude non-numeric or identifier columns from histogram plotting
cols_to_plot = [col for col in TARGET_COLS if col not in ['sample_id', 'image_path', 'State', 'target_name']]

for col in cols_to_plot:
    plt.figure(figsize=(8, 3)) # Create a new figure for each histogram
    plt.hist(df[col].dropna(), bins=50, edgecolor='black', alpha=0.7)
    plt.xlabel(col, fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.title(f'{col} Distribution', fontsize=14, fontweight='bold')
    plt.grid(alpha=0.3)
    plt.xticks(rotation=45, ha="right") # Rotate x-axis labels
    plt.tight_layout() # Adjust layout to prevent overlap
    plt.show()

In [ ]:
cols_to_plot = ['State', 'target_name']
n_rows, n_cols = 1, len(cols_to_plot)
fig, axes = plt.subplots(n_rows, n_cols, figsize=(4 * n_cols, 4 * n_rows))

# Ensure axes is an array even for a single subplot
axes = axes.flatten()

for ax, col in zip(axes, cols_to_plot):
    counts = df[col].value_counts()
    ax.pie(counts, labels=counts.index, autopct='%1.1f%%', startangle=140)
    ax.set_title(f'Distribution of {col}', fontsize=14, fontweight='bold')
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.tight_layout()
plt.show()

In [ ]:
# Convert 'Sampling_Date' to datetime objects
df['Sampling_Date'] = pd.to_datetime(df['Sampling_Date'])

# Extract the day of the year
df['Day_of_Year'] = df['Sampling_Date'].dt.dayofyear

# Calculate the correlation between 'target' and 'Day_of_Year'
correlation = df['target'].corr(df['Day_of_Year'])

print(f"The correlation between 'target' and 'Day_of_Year' is: {correlation}")

## Show sample images

In [ ]:
def show_images(df_sample, n=12, path_img=PATH_DATA):
    """Displays a linear sampling of images sorted by target value."""

    # Sort the DataFrame by the 'target' column
    df_sorted = df_sample.sort_values(by='target').reset_index(drop=True)

    # Perform linear sampling
    indices_to_show = np.linspace(0, len(df_sorted) - 1, n, dtype=int)
    df_to_show = df_sorted.iloc[indices_to_show]

    # Determine the number of rows and columns for subplots
    n_cols = 3  # You can adjust this number
    n_rows = (n + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 4 * n_rows))
    axes = axes.flatten()

    # Remove unused subplots if any
    for i in range(n, len(axes)):
        fig.delaxes(axes[i])

    for i, (idx, row) in enumerate(df_to_show.iterrows()):
        # Use image_path directly (includes train/ID....jpg)
        img_path = os.path.join(path_img, row['image_path'])

        if os.path.exists(img_path):
            img = Image.open(img_path).convert('RGB')
            axes[i].imshow(img)
            # Include the target value in the title
            title = f"ID: {row['sample_id']}\nTarget: {row['target']:.2f}"
            axes[i].set_title(title, fontsize=10)
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

# Example usage: Show 12 images linearly sampled based on target value
show_images(df, n=12)

# Dataset & DataModule

In [ ]:
from sklearn.preprocessing import LabelEncoder

class BiomassDataset(Dataset):
    """Simple dataset for biomass regression with metafeatures."""

    def __init__(self, df, path_img, transforms=None, mode='train', species_classes=None, target_name_classes=None):
        self.df = df.reset_index(drop=True)
        self.path_img = path_img
        self.transforms = transforms
        self.mode = mode
        self.target_col = 'target' if mode == 'train' else None
        self._len = len(self.df) * 2

        # Initialize LabelEncoders and fit
        self.target_name_encoder = LabelEncoder()
        # Use provided classes or create from DataFrame and Expose the classes as properties
        self.species_classes = species_classes or self.df['Species'].unique()
        self.target_name_classes = target_name_classes or self.df['target_name'].unique()
        # Prepare encoding
        self.target_name_encoder.fit(self.target_name_classes)
        self.df['target_name_encoded'] = self.target_name_encoder.transform(self.df['target_name'])

        # Encode categorical features within the dataset
        if self.mode != 'test':
            # Initialize LabelEncoders and fit
            self.species_encoder = LabelEncoder()
            self.species_encoder.fit(self.species_classes)
            self.df['Species_encoded'] = self.species_encoder.transform(self.df['Species'])

    def __len__(self):
        return self._len

    def __getitem__(self, idx):
        row = self.df.iloc[idx // 2]
        img_relative_path = row['image_path']
        img_path = os.path.join(self.path_img, img_relative_path)
        img = Image.open(img_path).convert('RGB')

        half = idx % 2
        width, height = img.size
        img_cropped = img.crop((0, 0, width // 2, height)) if half == 0 else img.crop((width // 2, 0, width, height))

        if self.transforms:
            img_cropped = self.transforms(img_cropped)

        target_name_encoded = torch.tensor(row['target_name_encoded'], dtype=torch.long) # Use long for categorical
            
        if self.mode == 'test':
            return img_cropped, target_name_encoded, img_relative_path

        # Extract and convert metafeatures to tensors (assuming they are already encoded in the dataframe)
        species_encoded = torch.tensor(row['Species_encoded'], dtype=torch.long) # Use long for categorical
        ndvi = torch.tensor(row['Pre_GSHH_NDVI'], dtype=torch.float32)
        height = torch.tensor(row['Height_Ave_cm'], dtype=torch.float32)
        target = torch.tensor(row[self.target_col], dtype=torch.float32)
        return img_cropped, target_name_encoded, target, species_encoded, ndvi, height

In [ ]:
# Initialize the dataset (assuming you have a DataFrame 'df' and image path 'PATH_DATA')
dataset = BiomassDataset(df, PATH_DATA)
# Get three random indices
random_indices = random.sample(range(len(dataset)), 3)

# Display the random samples
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, idx in enumerate(random_indices):
    img, target_name_encoded, target, species_encoded, ndvi, height = dataset[idx]
    # Convert the PyTorch tensor image back to PIL Image for displaying
    # This assumes the default tensor format from PILToTensor or similar
    if isinstance(img, torch.Tensor):
        img = img.permute(1, 2, 0).numpy() # Assuming CxHxW format, convert to HxWxD

    axes[i].imshow(img)
    # Display targets and other data in a multi-line title
    title = f"Target: {target:.2f}\n"
    title += f"Species: {dataset.species_classes[species_encoded]}\n"
    title += f"NDVI: {ndvi:.2f}\n"
    title += f"Height: {height:.2f}\n"
    title += f"Target Name: {dataset.target_name_classes[target_name_encoded]}"

    axes[i].set_title(title, fontsize=10)
    axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
from torchvision import transforms
from pathlib import Path


class BiomassDataModule(pl.LightningDataModule):
    """Simple DataModule for biomass regression with metafeatures."""

    def __init__(self, data_path, batch_size=32, img_size=(456, 456), val_split=0.2):
        super().__init__()
        self.save_hyperparameters()
        self.data_path = data_path
        self.batch_size = batch_size
        self.img_size = img_size
        self.val_split = val_split
        self.train_df: Optional[pd.DataFrame] = None
        self.val_df: Optional[pd.DataFrame] = None
        self.test_df: Optional[pd.DataFrame] = None
        self._color_mean = [0.485, 0.456, 0.406]
        self._color_std = [0.229, 0.224, 0.225]
        inimg_size = int(img_size[0] * 1.5)
        self.transforms = transforms.Compose([
            transforms.Resize((inimg_size, inimg_size)),
            transforms.RandomResizedCrop(self.img_size),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
            transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
            transforms.GaussianBlur(kernel_size=3),
            transforms.ToTensor(),
            transforms.Normalize(mean=self._color_mean, std=self._color_std),
        ])
        self.test_transforms = transforms.Compose([
            transforms.Resize((inimg_size, inimg_size)),
            transforms.CenterCrop(self.img_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=self._color_mean, std=self._color_std),
        ])
        self._num_workers = os.cpu_count() if os.cpu_count() is not None else 0

        # Load train and test data here to fit encoders on combined data
        self.df = pd.read_csv(os.path.join(self.data_path, 'train.csv'))

        # Get unique classes from the training data for consistent encoding
        self.species_classes = sorted(self.df['Species'].unique())
        self.target_name_classes = sorted(self.df['target_name'].unique())

    def setup(self, stage: Optional[str] = None):
        if stage == 'fit' or stage is None:
            shuffled_df = self.df.sample(frac=1, random_state=42).reset_index(drop=True)
            val_size = int(len(shuffled_df) * self.val_split)
            self.train_df = shuffled_df[:-val_size].copy() # Use .copy() to avoid SettingWithCopyWarning
            self.val_df = shuffled_df[-val_size:].copy() # Use .copy() to avoid SettingWithCopyWarning

        if stage == 'test' or stage is None:
            test_image_dir = os.path.join(self.data_path, 'test')
            assert os.path.isdir(test_image_dir)
            self.test_df = pd.read_csv(os.path.join(self.data_path, 'test.csv'))

    def train_dataloader(self):
        train_dataset = BiomassDataset(
            self.train_df,
            self.data_path,
            transforms=self.transforms,
            mode='train',
            species_classes=self.species_classes,
            target_name_classes=self.target_name_classes
        )
        return DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self._num_workers)

    def val_dataloader(self):
        val_dataset = BiomassDataset(
            self.val_df,
            self.data_path,
            transforms=self.test_transforms,
            mode='train',
            species_classes=self.species_classes,
            target_name_classes=self.target_name_classes
        )
        return DataLoader(val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self._num_workers)

    def test_dataloader(self):
        if self.test_df is None:
            self.setup(stage='test')

        test_dataset = BiomassDataset(
            self.test_df,
            self.data_path,
            transforms=self.test_transforms,
            mode='test',
            species_classes=self.species_classes,
            target_name_classes=self.target_name_classes
        )
        return DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=0)

    def teardown(self, stage: Optional[str] = None):
        # Clean up resources if needed
        pass

In [ ]:
# Example Usage (assuming df, PATH_DATA and TARGET_COLS are defined)
data_module = BiomassDataModule(PATH_DATA, batch_size=16, img_size=(448, 448))
data_module.setup()

# You can now access the dataloaders
train_loader = data_module.train_dataloader()
val_loader = data_module.val_dataloader()

print(f"Number of training batches: {len(train_loader)}")
print(f"Number of validation batches: {len(val_loader)}")

# Example of getting a batch (optional)
train_images, train_targets, train_species_encoded, train_ndvi, train_height, train_target_name_encoded = next(iter(train_loader))
print(f"Shape of training images batch: {train_images.shape}")
print(f"Shape of training targets batch: {train_targets.shape}")
print(f"Shape of training species encoded batch: {train_species_encoded.shape}")
print(f"Shape of training NDVI batch: {train_ndvi.shape}")
print(f"Shape of training height batch: {train_height.shape}")
print(f"Shape of training target name encoded batch: {train_target_name_encoded.shape}")

In [ ]:
# Get the first batch from the training dataloader
train_images, train_target_name_encoded, train_targets, train_species_encoded, train_ndvi, train_height = next(iter(train_loader))

# Determine how many images to show (e.g., the first 4 from the batch)
n_images_to_show = min(4, train_images.shape[0])

fig, axes = plt.subplots(1, n_images_to_show, figsize=(4 * n_images_to_show, 5))

# Ensure axes is an array even for a single image
if n_images_to_show == 1:
    axes = [axes]

# Assuming data_module is available in the environment to access class names
# If not, you might need to pass them from the data_module
# For now, assuming data_module is accessible
species_classes = data_module.species_classes
target_name_classes = data_module.target_name_classes

for i in range(n_images_to_show):
    img = train_images[i].permute(1, 2, 0).numpy() # Convert from CxHxW to HxWxD for displaying
    # Denormalize the image for better visualization (using ImageNet standards)
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean
    img = np.clip(img, 0, 1) # Clip values to be between 0 and 1

    axes[i].imshow(img)
    # Display targets and other data in a multi-line title
    title = f"Target: {train_targets[i].item():.2f} [{train_targets.dtype}]\n"
    title += f"Species: {train_species_encoded[i]} [{train_species_encoded.dtype}]\n"
    title += f"NDVI: {train_ndvi[i].item():.2f} [{train_ndvi.dtype}]\n"
    title += f"Height: {train_height[i].item():.2f} [{train_height.dtype}]\n"
    title += f"Target Name: {train_target_name_encoded[i]} [{train_target_name_encoded.dtype}]"

    axes[i].set_title(title, fontsize=10)
    # axes[i].axis('off')

plt.tight_layout()
plt.show()

# LightningModule & training

to select backbones: https://github.com/huggingface/pytorch-image-models/blob/main/results/results-imagenet-a-clean.csv

In [ ]:
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR
import torchmetrics


class SimplifiedBiomassModel(pl.LightningModule):
    """Simplified regression model predicting NDVI, Height, and Target from image."""

    def __init__(
        self,
        model_name="resnet18",
        pretrained=True,
        learning_rate=5e-4,
        loss_weight_smooth_l1=0.5,
        loss_weight_ndvi=1.0,  # Add weight for NDVI loss
        loss_weight_height=1.0, # Add weight for Height loss
        loss_weight_target=2.0, # Add weight for main target loss (higher by default)
        T_max=10, # Number of epochs for CosineAnnealingLR
        num_target_names=5, # Add number of unique target names
        hidden_dims=[256, 128] # Change hidden_dim to be a list of dimensions
    ):
        super().__init__()
        self.save_hyperparameters()
        # Ensure hidden_dims is a list even if a single value is passed
        if not isinstance(hidden_dims, list):
            hidden_dims = [hidden_dims]
        self.hparams.hidden_dims = hidden_dims # Save the list in hparams

        self.backbone = timm.create_model(model_name, pretrained=pretrained, num_classes=0, global_pool='avg')
        in_features = self.backbone.num_features

        # Dynamically build the regression head based on hidden_dims list
        layers = []
        current_dim = in_features + num_target_names
        for h_dim in self.hparams.hidden_dims:
            layers.append(nn.Linear(current_dim, h_dim))
            layers.append(nn.ReLU()) # Use ReLU activation
            current_dim = h_dim
        # Add the final output layer
        layers.append(nn.Linear(current_dim, 3)) # Output 3 values: NDVI, Height, Target
        self.regression_head = nn.Sequential(*layers)

        # Loss functions
        self.smooth_l1_criterion = nn.SmoothL1Loss()
        self.mse_criterion = nn.MSELoss()

        # Use MetricCollection for train and validation metrics
        metrics = torchmetrics.MetricCollection({
            'ndvi_mae': torchmetrics.MeanAbsoluteError(),
            'height_mae': torchmetrics.MeanAbsoluteError(),
            'target_mae': torchmetrics.MeanAbsoluteError(),
            'target_mse': torchmetrics.MeanSquaredError(),
        })

        self.train_metrics = metrics.clone(prefix='train_')
        self.val_metrics = metrics.clone(prefix='val_')


    def forward(self, x, target_name_encoded):
        # Pass the input image through the backbone
        features = self.backbone(x)
        # One-hot encode the target_name_encoded
        target_name_one_hot = F.one_hot(target_name_encoded, num_classes=self.hparams.num_target_names).float()
        # Concatenate image features and one-hot encoded target name
        combined_features = torch.cat([features, target_name_one_hot], dim=1)
        # Pass the combined features through the regression head
        predictions = self.regression_head(combined_features) # Output is [batch_size, 3]
        return predictions

    def _compute_loss(self, predictions, targets):
        """Computes the combined loss for NDVI, Height, and Target."""
        # Unpack predictions and targets
        ndvi_pred, height_pred, target_pred = predictions[:, 0], predictions[:, 1], predictions[:, 2]
        ndvi_target, height_target, target_target = targets # Targets tuple contains ndvi, height, target

        # Calculate individual losses with weights
        # Normalize weights to be in the range 0-1 while maintaining relative proportions
        total_weight = self.hparams.loss_weight_ndvi + self.hparams.loss_weight_height + self.hparams.loss_weight_target
        normalized_ndvi_weight = self.hparams.loss_weight_ndvi / total_weight
        normalized_height_weight = self.hparams.loss_weight_height / total_weight
        normalized_target_weight = self.hparams.loss_weight_target / total_weight

        ndvi_loss = self.mse_criterion(ndvi_pred.squeeze(), ndvi_target.squeeze()) * normalized_ndvi_weight
        height_loss = self.mse_criterion(height_pred.squeeze(), height_target.squeeze()) * normalized_height_weight

        # Main target loss with weighted SmoothL1 and MSE
        smooth_l1_loss = self.smooth_l1_criterion(target_pred.squeeze(), target_target.squeeze())
        mse_loss = self.mse_criterion(target_pred.squeeze(), target_target.squeeze())
        main_target_loss = (self.hparams.loss_weight_smooth_l1 * smooth_l1_loss + (1 - self.hparams.loss_weight_smooth_l1) * mse_loss) * normalized_target_weight

        # Combine losses
        total_loss = ndvi_loss + height_loss + main_target_loss
        return total_loss, ndvi_loss, height_loss, main_target_loss

    def training_step(self, batch, batch_idx):
        # The batch now contains images, target_name_encoded, target, species, ndvi, height
        images, target_name_encoded, target, species, ndvi, height = batch
        predictions = self(images, target_name_encoded) # Forward pass with images and target_name_encoded

        # Pack the relevant targets
        targets = (ndvi, height, target)
        total_loss, ndvi_loss, height_loss, main_target_loss = self._compute_loss(predictions, targets)

        # Log losses
        self.log('train_total_loss', total_loss, on_step=True, prog_bar=True)
        self.log('train_ndvi_loss', ndvi_loss, on_step=True, prog_bar=False)
        self.log('train_height_loss', height_loss, on_step=True, prog_bar=False)
        self.log('train_main_target_loss', main_target_loss, on_step=True, prog_bar=False)

        # Log metrics using MetricCollection
        metrics_preds = torch.stack([predictions[:, 0], predictions[:, 1], predictions[:, 2]], dim=1)
        metrics_targets = torch.stack([ndvi, height, target], dim=1)

        # Update the MetricCollection directly
        self.train_metrics.update(metrics_preds, metrics_targets)

        self.log_dict(self.train_metrics, on_step=True, on_epoch=True, prog_bar=False)
        return total_loss

    def validation_step(self, batch, batch_idx):
        # The batch structure from BiomassDataset in 'train' mode is:
        # img_cropped, target_name_encoded, target, species_encoded, ndvi, height
        images, target_name_encoded, target, species_target, ndvi_target, height_target = batch
        predictions = self(images, target_name_encoded) # Forward pass with images and target_name_encoded

        # Pack the relevant targets
        targets = (ndvi_target, height_target, target)
        total_loss, ndvi_loss, height_loss, main_target_loss = self._compute_loss(predictions, targets)

        # Log losses
        self.log('val_total_loss', total_loss, on_step=True, prog_bar=False)
        self.log('val_ndvi_loss', ndvi_loss, on_step=True, prog_bar=False)
        self.log('val_height_loss', height_loss, on_step=True, prog_bar=False)
        self.log('val_main_target_loss', main_target_loss, on_step=True, prog_bar=False)

        # Log metrics using MetricCollection
        metrics_preds = torch.stack([predictions[:, 0], predictions[:, 1], predictions[:, 2]], dim=1)
        metrics_targets = torch.stack([ndvi_target, height_target, target], dim=1)

        # Update the MetricCollection directly
        self.val_metrics.update(metrics_preds, metrics_targets)
        self.log_dict(self.val_metrics, on_step=True, prog_bar=False)


    def predict_step(self, batch, batch_idx):
        """Prediction step for the test set."""
        # The batch structure from BiomassDataset in 'test' mode is:
        # img_cropped, target_name_encoded, img_relative_path
        images, target_name_encoded, img_relative_path = batch
        predictions = self(images, target_name_encoded) # Forward pass with images and target_name_encoded
        # Unpack predictions
        ndvi_pred, height_pred, target_pred = predictions[:, 0], predictions[:, 1], predictions[:, 2]
        # Return all predictions and the original image path
        return target_pred.squeeze(), img_relative_path

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.hparams.learning_rate)
        scheduler = {
            'scheduler': CosineAnnealingLR(optimizer, T_max=self.hparams.T_max),
            'interval': 'epoch',
            'frequency': 1,
        }
        return [optimizer], [scheduler]

# Initialize the simplified model
# You need to get the number of unique target names from your data_module
# Assuming data_module is already initialized and setup has been called
num_target_names = len(data_module.target_name_classes)

model = SimplifiedBiomassModel(
    model_name="efficientnet_b5.sw_in12k_ft_in1k",
    pretrained=False,
    learning_rate=1e-2,
    num_target_names=num_target_names, # Pass the number of target names
    hidden_dims=[256, 128, 64], # Example: three hidden layers with dimensions 512, 256, 128
    T_max=10, # Set T_max to the number of epochs for CosineAnnealingLR
    loss_weight_ndvi=0.5, # Example weight for NDVI loss
    loss_weight_height=0.5, # Example weight for Height loss
    loss_weight_target=2.0 # Example weight for main target loss
)

In [ ]:
from pytorch_lightning.loggers import CSVLogger

# Initialize the CSVLogger
logger = CSVLogger("logs", name="biomass_regression")

# Initialize the Trainer
trainer = pl.Trainer(
    max_epochs=75, # You can adjust the number of epochs
    logger=logger,
    accelerator='auto', # Use auto to automatically select accelerator (GPU/CPU)
    devices='auto', # Use auto to automatically select devices
    precision='16-mixed', # Use Automatic Mixed Precision (AMP)
    log_every_n_steps=5, # Update progress bar every 5 steps
    # gradient_clip_val=1.0, # Add gradient clipping to prevent NaN
    accumulate_grad_batches=6,
)

# Fit the model
trainer.fit(model, data_module)

In [ ]:
# Define the path to save the model
model_save_path = "biomass_regression_model.pth"

# Save the model's state dictionary
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Read the metrics.csv using the trainer's logger directory
metrics = pd.read_csv(f"{trainer.logger.log_dir}/metrics.csv")

# Remove the step column and set epoch as index
# metrics.set_index("step", inplace=True)
display(metrics.dropna(axis=1, how="all").head())

# Melt the DataFrame to long-form for plotting
metrics_melted = metrics.reset_index().melt(id_vars='epoch', var_name='metric', value_name='value')

# Define metric groups
metric_groups = {
    'Loss': [c for c in metrics.columns if "_loss" in c],
    'MAE': [c for c in metrics.columns if "_mae" in c and "loss" not in c],
    'MSE': [c for c in metrics.columns if "_mse" in c and "loss" not in c],
}

# Plot metrics for each group in a separate chart
for title, metric_list in metric_groups.items():
    # Filter melted DataFrame for the current group
    group_metrics = metrics_melted[metrics_melted['metric'].isin(metric_list)]

    plt.figure(figsize=(10, 5))
    sns.lineplot(data=group_metrics, x='epoch', y='value', hue='metric')
    plt.title(f'{title} over Epochs', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel(title, fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.yscale('log')  # Set y-axis to logarithmic scale
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left') # Move legend outside and to the right
    plt.tight_layout() # Adjust layout to prevent legend overlap
    plt.show()

# Prediction & submission

In [ ]:
# Assuming data_module is already initialized and setup has been called for the test stage
# If not, uncomment and run the DataModule initialization and setup cells first
# data_module = BiomassDataModule(PATH_DATA, batch_size=64)
# data_module.setup(stage='test')

# Get the test dataloader
test_loader = data_module.test_dataloader()

# Get the first batch from the test dataloader
test_images, test_target_names, test_img_paths = next(iter(test_loader))

# Determine how many images to show (e.g., the first 4 from the batch)
n_images_to_show = min(4, test_images.shape[0])

fig, axes = plt.subplots(1, n_images_to_show, figsize=(4 * n_images_to_show, 5))

# Ensure axes is an array even for a single image
if n_images_to_show == 1:
    axes = [axes]

for i in range(n_images_to_show):
    img = test_images[i].permute(1, 2, 0).numpy() # Convert from CxHxW to HxWxD for displaying
    # Denormalize the image for better visualization (using ImageNet standards)
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean
    img = np.clip(img, 0, 1) # Clip values to be between 0 and 1

    axes[i].imshow(img)
    # Display the sample_id for test images
    axes[i].set_title(f"path: {test_img_paths[i]}\n target name: {test_target_names[i]}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Need to re-initialize the test dataloader to get the sample_ids in order
test_loader = data_module.test_dataloader()

# Debugging: Print the type and content of test_loader
print(f"Type of test_loader: {type(test_loader)}")
print(f"Content of test_loader: {test_loader}")


# Generate predictions on the test set by manually iterating through the dataloader
all_predictions = []
all_image_paths = []

model.eval() # Set the model to evaluation mode
with torch.no_grad(): # Disable gradient calculation
    for batch in test_loader:
        # Assuming predict_step returns (target_pred, species_pred, ndvi_pred, height_pred, img_relative_path)
        target_pred, img_relative_path = model.predict_step(batch, batch_idx=0) # Pass batch and dummy batch_idx

        # Convert target predictions to list and extend the all_predictions list
        all_predictions.extend(target_pred.tolist())
        all_image_paths.extend(img_relative_path)


# Create a DataFrame with predictions and image paths
predictions_raw_df = pd.DataFrame({'image_path': all_image_paths, 'target': all_predictions})

# Display the first few rows of the submiss
print("Submission DataFrame head:")
display(predictions_raw_df.head())

# You can save the submission_df to a CSV file in the required format
# submission_df.to_csv('submission.csv', index=False)

In [ ]:
# prevent any negative values
predictions_raw_df[predictions_raw_df["target"] < 0]["target"] = 0

# Group by image_path and take the mean of the predictions
prediction_df = predictions_raw_df.groupby('image_path')['target'].mean().reset_index()

In [ ]:
# Define the path to the sample submission file
test_csv_path = os.path.join(PATH_DATA, 'test.csv')

# Load the sample submission file
test_csv = pd.read_csv(test_csv_path)
# display(test_csv.head())

# del sample_submission_df['target']
test_csv = test_csv.merge(prediction_df, on='image_path', how='left')
display(test_csv.head())

# dump prediction into CSV file
test_csv[["sample_id", "target"]].to_csv('submission.csv', index=False)

In [ ]:
! head submission.csv